
# Instructions



This is a Jupyter notebook. A notebook contains cells, some of them
contain just text, such as this one that you are reading right
now. The other important type of cell in a notebook is a code
cell. You can run code in the python programming language that is
contained in a code cell. Code cells look like this:



In [1]:
55/34

In [1]:
(1+5**(1/2))/2 # notice that x**y means x to the power of y in python
# notice also that comments in python start with a hash (#)

By clicking over a code cell, and then pressing the 'shift' key and
the 'enter' key together, you can run that particular code cell and
advance to next cell. Try to run the code contained in the first code
cell above.

The result of the division of 55 over 34 should appear just below the
code. Try now to run the next code cell (no need to click over the
code cell, it is already selected).

If you are in a rush and do not want to learn much about the python
programming language and just care about observing the results of the
different models you can skip to Section [Model 1, just the necessary code](#Model-1,-just-the-necessary-code)
and just advance cell by cell by pressing Shift+Enter.

Other ways to run a code cell are to click in the "Cell" menu and
select "Run Cells", or to use the "Run" button that is below the
"Cell" menu. If you are interested in learning more about the Jupyter
notebook interface you can click in the "Help" menu and select "User
interface Tour". However, pressing Shift+Enter for each cell in this
tutorial is all you will need to do.

Notice that the first code cell starts with "In [1]:" and its result
with "Out[1]:". The second code cell, starts with "In [2]:" and its
result with "Out[2]:".

You can also modify any code cell and re-run it to obtain a different
result. Try to modify the first cell and calculate the result of the
division of 89 over 55. In that case the you should see now that the
code cell starts with "In [3]:" and its result with "Out[3]:", meaning
that it is the third evaluation that you have done.




# Preliminaries: Vectors in python



To represent and do operations with vectors in python we will use the
NumPy library. A library in python is a collection of methods and
constants.

Let's start by importing the library:



In [1]:
import numpy

Now we have access to all the methods that the NumPy library
provides. For example NumPy provides the `numpy.cos` method that
computes the cosine of a number:



In [1]:
numpy.cos(0)

it also provides the number pi as the constant `numpy.pi`:



In [1]:
numpy.cos(numpy.pi)

An array is the most basic element of the NumPy library and we will
represent vectors and matrices as arrays. For example:



In [1]:
numpy.array([1, 1, 1])

represents the vector $(1, 1, 1)$ in $\mathbb{R}^3$, whereas:



In [1]:
numpy.array([[1, 0], [0, 1]])

represents the 2x2 identity matrix.

We will use the NumPy method `linspace` in a little bit, so let's
check out how it works:



In [1]:
numpy.linspace(25, 100, 4)

The method takes as arguments three numbers `x, y, n` and gives back
an array with `n` numbers equally spaced between `x` and `y`.

Let's store a vector in a variable:



In [1]:
v = numpy.linspace(25, 100, 4)

Notice that this code cell did not output anything when it is
run. However, now we have the variable `v` available, and it has the
content of the result of `numpy.linspace(25, 100, 4)`, we can call it
back whenever we want:



In [1]:
v

To access the a specific entry of a vector we use square brackets and
indicate the number of the element of the array that we want to
access:



In [1]:
v[1]

Notice that the numbering in python starts with 0, so this will be
the first entry of the vector `v`:



In [1]:
v[0]

Therefore this code cell should give you an error:



In [1]:
v[4]

To access the last entry of `v` you should use:



In [1]:
v[3]


# Solving an ODE model



An system of ordinary differential equations (ODE) can be written in
its most general form as:

$$\frac{\mathrm{d}}{\mathrm{d}t}\mathbf{x}(t)
= \begin{bmatrix}\frac{\mathrm{d}}{\mathrm{d}t} x_1(t)\\ \vdots
\\ \frac{\mathrm{d}}{\mathrm{d}t} x_n(t) \end{bmatrix} = f(\mathbf{x}(t), t,
\mathbf{z}),$$

where $\mathbf{x}(t)$ and $\mathbf{z}$ are vectors and $f$ is a
function that maps to a vector of the same dimension as
$\mathbf{x}(t)$. We also have to provide an initial condition for the
system:

$$\mathbf{x}(0) = \mathbf{x_0}.$$

For us, the vector $\mathbf{x}(t)$ represents all the classes in our
model at a particular time $t$, and the vector $\mathbf{z}$ represents
all the parameters in our model.

To solve an ODE in Python we have to define the function $f$ as a
python function and provide the initial condition $\mathbf{x_0}$.




# Model 1: continuous-time endemic



Let's start with Model 1, which has the equations

$$\mathbf{x} = \begin{bmatrix} S \\ I \\ R \end{bmatrix}$$

$$\frac{\mathrm{d}}{\mathrm{d}t}\mathbf{x}(t)
= \begin{bmatrix}\frac{\mathrm{d}}{\mathrm{d}t} S(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} I(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} R(t) \end{bmatrix}
= \begin{bmatrix} (1-p) \mu N(t)  + \omega R(t) - \big( \frac{a \beta
I(t)}{N(t)} + \mu \big) S(t)
\\ \frac{a \beta I(t)}{N(t)} S(t) - (\gamma + \mu) I(t) \\ p \mu N(t)
\gamma I(t) - (\mu + \omega) R(t)
\end{bmatrix}$$

where $N(t)=S(t)+I(t)+N(t)$. We also have to determine an initial
condition deciding how many individuals susceptible, infected and
recovered at the start of the epidemic ($S(0)$, $I(0)$, and $R(0)$,
respectively).

As stated before, we need to define the function $f$ in python to
obtain the solution of the ODE.




## Defining a function in python



Function definitions in python start with `def` then have the function
name that you want to define, all the arguments that the function
takes in parenthesis and separated by a comma, and a semicolon
(:). The code of the function is indented and the directive `return`
determines what will be returned when evaluating the function. Let's
define that for Model 1:



In [1]:
def f_model_1(x, t, beta, a, mu, omega, gamma, p):
    S = x[0]
    I = x[1]
    R = x[2]

    N = S + I + R

    dS_dt = (1-p)*mu*N + omega*R - (a*beta*I/N - mu)*S
    dI_dt = (a*beta*I/N)*S - (gamma + mu)*I
    dR_dt = p*mu*N + gamma*I - (omega+mu)*R

    return numpy.array([dS_dt, dI_dt, dR_dt])

Notice that just as we described above for the general form of an ODE
system the function takes `x`, which is a vector (represented as a
NumPy array), then `t` which is the time, and then the parameters of
the model, in our case `beta`, `a`, `mu`, `omega`, `gamma` and `p`. The
function returns another vector (again represented as a NumPy array)
of the same dimension as `x` (3, because we have three classes).




## Solving ODE systems in python



To obtain the solution of an ODE system in python we need a method to
solve ODE systems. The method that we will use is part of the SciPy
library and it is contained within the module `integrate`.  A module
in python is a subset of a library, it is just a division of a library
that is used when a library is very big, as it is the case for the
SciPy library. Because the method `odeint` is the only method of the
whole SciPy library that we need, we can import just that method:



In [1]:
from scipy.integrate import odeint

The method `odeint` takes as arguments a function (the $f$ in the ODE
general form), an initial condition (the vector $\mathbf{x}_0$), the
times for which we want to obtain the results, and the extra arguments
of the function (the parameters $\mathbf{z}$) in parenthesis and
separated by commas.

So suppose that you want to know the solution of Model 1 for 10 days
and 100 days after the start of the outbreak ($t=10$ and $t=100$) when
the parameters are $\beta=0.05$, $a=10$, $\mu=0.001$, $\omega=0$,
$\gamma=0.2$, and $p=0$ and given the initial condition $S(0)=99$,
$I(0)=1$, $R(0)=0$ (in other words a population of 100 where one
individual is infected and the rest are susceptible). Then you would
use the method `odeint` like this:



In [1]:
odeint(f_model_1, [99, 1, 0], [10, 100], (0.05, 10, 0.001, 0, 0.2, 0))

Notice that the result is a matrix with two rows and three
columns. The columns represent the values of $S$, $I$ and $R$ and the
rows represent the times for which we wanted the results ($t=10$ and
$t=100$ in this case). Numbers are by default represented in
exponential notation, `1.60361708e+01` means $1.60361708 \times 10^1 =
16.0361708$, whereas, `2.73164326e-03` means $2.73164326 \times
10^{-3}$.

To be able to explore the dynamics of any model, we obviously need
more points in time. That is why the NumPy method `linspace` is
helpful. For example we can output all the results for the first 100
days of the epidemic with a resolution of half a day by asking for 200
values of $t$ between $t=0$ and say $t=100$:



In [1]:
odeint(f_model_1, [99, 1, 0], numpy.linspace(0, 100, 200), (0.05, 10, 0.001, 0, 0.2, 0))

The result now has 200 rows (and still 3 rows for $S$, $I$, $R$).

Let us save a little bit of code writing by defining a new function
that solves Model 1 for all times between $t=0$ and an arbitrary
maximum time, with a default resolution of 200 times:



In [1]:
default_resolution = 200

def model_1(beta, a, mu, omega, gamma, p, initial_S, initial_I, initial_R, max_time):

    ts = numpy.linspace(0, max_time, default_resolution)
    x0 = [initial_S, initial_I, initial_R]
    xs = odeint(f_model_1, x0, ts, (beta, a, mu, omega, gamma, p))

    return xs

Now if we want the store the results of Model 1 up to time $t=365$ (a
year), for the parameters $\beta=0.05$, $a=10$, $\mu=4\times10^{-5}$,
$\omega=0.01$, $\gamma=0.2$, $p=0$ given the initial condition
$S(0)=99$, $I(0)=1$, $R(0)=0$ we can call the function that we just
defined:



In [1]:
results_model_1 =  model_1(0.05, 10, 4*10**(-5), 0.01, 0.2, 0, 99, 1, 0, 365)
results_model_1

The variable `results_model_1` contains now the result of the ODE
system. If we want to change the results for example for $\beta=0.04$, then we
can evaluate:



In [1]:
results_model_1 = model_1(0.04, 10, 4*10**(-5), 0.01, 0.2, 0, 99, 1, 0, 365)
results_model_1

Notice that `default_resolution` is just a variable, and we can change
it whenever we want and that will affect the results obtained when the
function `model_1` is called. Let's check that using the method NumPy
method `shape` to see the dimensions of the NumPy arrays obtained:



In [1]:
numpy.shape(results_model_1)

That is, `results_model_1` has 200 rows and 3 columns. Now if we
change the variable `default_resolution`, we obtain 1,000 rows and 3
columns:



In [1]:
default_resolution = 1000
results_model_1 = model_1(0.04, 10, 4*10**(-5), 0.01, 0.2, 0, 99, 1, 0, 365)
numpy.shape(results_model_1)

Obviously, it is not very helpful to see results as a 1,000 by 3 matrix
of numbers, so we will visualize results as graphs.




## Plotting



To plot results we need access to another library, Matplotlib. We will
use several methods from its module `pyplot`, so let's import the
whole module:



In [1]:
from matplotlib import pyplot

Now all the methods of the module `pyplot` are available.

The first two methods that we will need are `pyplot.plot` and
`pyplot.show`, that unsurprisingly create a plot and show the
plot. `pyplot.plot` needs to be provided with the x-values (for our
case values of $t$) and y-values (for our case values of $S(t)$,
$I(t)$, and $R(t)$). y-values have to be of the same size as the
x-values, but we can plot several curves in the same graph by giving
the y-values in a matrix form, as long as the number of rows in that
matrix is the same as the dimension of the x-values. Luckily, this is
exactly how we obtained our results:



In [1]:
pyplot.plot(numpy.linspace(0, 365, default_resolution), results_model_1)
pyplot.show()

Plots are not shown by default after being created because we might
want to add more elements to a plot before showing it. For example, we
can make add a label for the x-axis and legends that help us
distinguished the classes in the model:



In [1]:
pyplot.plot(numpy.linspace(0, 365, default_resolution), results_model_1)

pyplot.xlabel("Time (days)")

pyplot.legend(["S", "I", "R"])

pyplot.show()

We will be plotting a lot, so let's define a function that we can call
later to save some time:



In [1]:
def plot_results(ts, xs, legends):
    pyplot.plot(ts, xs)
    pyplot.legend(legends)
    pyplot.xlabel("Time (days)")
    pyplot.ylim([0, numpy.sum(xs[0])]) # this sets the y-axis limit to the total population
    pyplot.show()

Now we just need to call `plot_results`, instead of the four lines of
code of the previous to last code cell:



In [1]:
plot_results(numpy.linspace(0, 365, default_resolution), results_model_1, ["S", "I", "R"])


## Setting default values



It is cumbersome to have to write the value of each of the parameters
of Model 1, whenever we want to call `model_1`, and besides that we
need to be careful with the order in which we provide the parameter
values to the function `model_1`. We can avoid that problem by
defining default values for the function `model_1`. Let's redefine the
`model_1` function to incorporate default values, and also let's go
further plotting the results and printing the value of $\mathcal{R}_0$
each time that we call the function `model_1`:



In [1]:
def model_1(beta=0.05,
            a=10.0,
            mu_times_365=0.000,
            omega=0.00,
            one_over_gamma=5.0,
            p=0,
            initial_S=99,
            initial_I=1,
            initial_R=0,
            max_time=365.0):

    mu = mu_times_365/365
    gamma = 1/one_over_gamma

    ts = numpy.linspace(0, max_time, default_resolution)
    x0 = [initial_S, initial_I, initial_R]
    xs = odeint(f_model_1, x0, ts, (beta, a, mu, omega, gamma, p))

    plot_results(ts, xs, ["S (susceptible)", "I (infectious)", "R (recovered)"])

    R0 = a*beta/(1/one_over_gamma+mu)
    print("R0 = {:.2f}".format(R0))
    # this means print 'R0 = ' and then the value of the variable R0 as a float rounding to two digits

    return xs

Notice that we defined `model_1` to take $1/\gamma$ not $\gamma$,
since $1/\gamma$ is the average duration of infectiousness, so it is
more natural to talk about it, instead of the recovery rate. Also the
range of $\mu$ is quite small so we will giving instead $\mu \times
365$ (why 365? what is $1/\mu$ if the time unit is days?).

Now we can call the function `model_1` without parameters, and the
default ones will be used:



In [1]:
model_1()

We can also change just a couple of parameters and we do not need to
worry about the order, so long as we tell exactly which ones are we changing:



In [1]:
model_1(mu_times_365=1/70, beta=0.06)


## Interacting with the parameters



`interact` is the last method from another library that we will need:



In [1]:
from ipywidgets import interact

Let's see how it works:



In [1]:
interact(model_1,
         beta=(0.01, 0.1, 0.001),
         a=(5, 15, 0.1),
         mu_times_365=(0.0, 1/20, 1/100),
         omega=(0, 1, 0.05),
         one_over_gamma=(2.0, 30.0, 0.1),
         p=(0, 1, 0.01),
         initial_S=(0, 1000),
         initial_I=(0, 1000),
         initial_R=(0, 1000),
         max_time=(1, 365*5, 30))

Notice that the arguments that we give to `interact` are the name of a
function to interact with, in our case `model_1`, and then each of the
arguments of that function in the form of 3 numbers: an initial value,
a final value, and a step value. We can also just provide 2 numbers,
as we did above for `initial_I` and `initial_R`, and the step value
will be equal to 1 by default.

Now you can change the values of the parameters very fast with the
sliders, and see how they impact the results of the model.

Since we will be using the same parameter range for the rest of the
models, there is a nice trick that we can use by defining what is
called a dictionary:



In [1]:
default_sliders = {'beta':(0.01, 0.2, 0.001),
                   'a':(5, 15, 0.25),
                   'mu_times_365':(0.0, 1/20, 1/100),
                   'omega':(0, 1, 0.05),
                   'one_over_gamma':(2.0, 30.0, 0.5),
                   'p':(0, 1, 0.01),
                   'initial_S':(0, 1000),
                   'initial_I':(0, 1000),
                   'initial_R':(0, 1000),
                   'max_time':(1, 365*5, 30)}

And now if we pass `**default_sliders` to any function, python will
substitute that part with `beta=(0.01, 0.1, 0.001)`, `a=(5, 15, 0.1)`,
etc. So this is exactly the same call that we had before:



In [1]:
interact(model_1, **default_sliders)


## Model 1, just the necessary code



$$\mathbf{x} = \begin{bmatrix} S \\ I \\ R \end{bmatrix}$$

$$\frac{\mathrm{d}}{\mathrm{d}t}\mathbf{x}(t)
= \begin{bmatrix}\frac{\mathrm{d}}{\mathrm{d}t} S(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} I(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} R(t) \end{bmatrix}
= \begin{bmatrix} (1-p) \mu N(t)  + \omega R(t) - \big( \frac{a \beta
I(t)}{N} -\mu \big) S(t)
\\ \frac{a \beta I(t)}{N} S(t) - (\gamma + \mu) I(t) \\ p \mu N(t)
\gamma I(t) - (\mu + \omega) R(t)
\end{bmatrix}$$



In [1]:
import numpy
from scipy.integrate import odeint
from matplotlib import pyplot
from ipywidgets import interact


def plot_results(ts, xs, legends):
    pyplot.plot(ts, xs)
    pyplot.legend(legends)
    pyplot.xlabel("Time (days)")
    pyplot.ylim([0, numpy.sum(xs[0])])
    pyplot.show()


def f_model_1(x, t, beta, a, mu, omega, gamma, p):
    S = x[0]
    I = x[1]
    R = x[2]

    N = S + I + R

    dS_dt = (1-p)*mu*N + omega*R - (a*beta*I/N - mu)*S
    dI_dt = (a*beta*I/N)*S - (gamma + mu)*I
    dR_dt = p*mu*N + gamma*I - (omega+mu)*R

    return numpy.array([dS_dt, dI_dt, dR_dt])


default_resolution = 1000


def model_1(beta=0.05,
            a=10.0,
            mu_times_365=0.00,
            omega=0.00,
            one_over_gamma=5.0,
            p=0,
            initial_S=99,
            initial_I=1,
            initial_R=0,
            max_time=365.0):

    mu = mu_times_365/365
    gamma = 1/one_over_gamma

    ts = numpy.linspace(0, max_time, default_resolution)
    x0 = [initial_S, initial_I, initial_R]
    xs = odeint(f_model_1, x0, ts, (beta, a, mu, omega, gamma, p))

    plot_results(ts, xs, ["S (susceptible)", "I (infectious)", "R (recovered)"])

    R0 = a*beta/(gamma+mu)
    print("R0 = {:.2f}".format(R0))

    return xs


default_sliders = {'beta':(0.01, 0.2, 0.001),
                   'a':(5, 15, 0.25),
                   'mu_times_365':(0.0, 1/20, 1/100),
                   'omega':(0, 1, 0.01),
                   'one_over_gamma':(2.0, 30.0, 0.5),
                   'p':(0, 1, 0.01),
                   'initial_S':(0, 1000),
                   'initial_I':(0, 1000),
                   'initial_R':(0, 1000),
                   'max_time':(1, 365*5, 30)}


interact(model_1, **default_sliders)


# Model 2: continuous-time epidemic



In Model 2 we do not account for births and deaths:

$$\frac{\mathrm{d}}{\mathrm{d}t}\mathbf{x}(t)
= \begin{bmatrix}\frac{\mathrm{d}}{\mathrm{d}t} S(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} I(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} R(t) \end{bmatrix}
= \begin{bmatrix} \omega R(t) - \big(\frac{a \beta I(t)}{N(t)} + \nu\big) S(t)
\\ \frac{a \beta I(t)}{N(t)} S(t) - \gamma I(t) \\ \gamma
I(t) + \nu S(t)- \omega R(t)
\end{bmatrix}$$

but we assume that we will immunize individuals from the susceptible
class at a rate of $\nu$.

As we learned from Model 1, we just need to define the function $f$,
and another function that calls `odeint` to solve the system and that
plots the results using `plot_results`:



In [1]:
def f_model_2(x, t, beta, a, omega, gamma, nu):
    S = x[0]
    I = x[1]
    R = x[2]

    N = S + I + R
    dS_dt = omega*R - (a*beta*I/N + nu)*S
    dI_dt = (a*beta*I/N)*S - gamma*I
    dR_dt = gamma*I + nu*S - omega*R

    return numpy.array([dS_dt, dI_dt, dR_dt])


def model_2(beta=0.05,
            a=10,
            omega=0.00,
            one_over_gamma=5.0,
            p=0.0,
            VE=0.2,
            initial_S=99.0,
            initial_I=1.0,
            initial_R=0.0,
            max_time=100.0):

    nu = -numpy.log(1-p*VE)
    gamma = 1/one_over_gamma

    ts = numpy.linspace(0, max_time, default_resolution)
    x0 = [initial_S, initial_I, initial_R]
    xs = odeint(f_model_2, x0, ts, (beta, a, omega, gamma, nu))

    plot_results(ts, xs, ["S (susceptible)", "I (infectious)", "R (recovered)"])

    R0 = a*beta/gamma
    print("R0 = {:.2f}".format(R0))

    return xs

Now we can call `model_2` with the default parameters:



In [1]:
model_2()

We can use `interact` again to see how the model behaves when the
parameters change. Because we already defined some default ranges for
the parameters when we were interacting with Model 1, we can simply
call:



In [1]:
interact(model_2, **default_sliders)

Notice however that the vaccine efficacy variable $\mathrm{VE}$ has a
strange range, `interact` allowed to set a negative value. Think about
what why this variable cannot be negative.

The problem is that when we defined `default_sliders`, we did not
include $\mathrm{VE}$, because Model 1 did not have that as a
parameter. Let's check the variable `default_sliders` again:



In [1]:
default_sliders

Notice that it contains ranges for all arguments that the function
`model_2` takes, except for $\mathrm{VE}$. We can fix it easily,
without having to write again `default_sliders`, we just need to add
another item:



In [1]:
default_sliders['VE'] = (0, 1, 0.05)

Now `default_sliders` has also the right range for $\mathrm{VE}$:



In [1]:
default_sliders

and `interact` will work as wanted:



In [1]:
interact(model_2, **default_sliders)


## Computing the final size in Model 2



We can modify model 2 to find the final size of the epidemic, we do
this by keeping track of the cumulative infections ($C$) using the
equations:

$$\frac{\mathrm{d}}{\mathrm{d}t}\mathbf{x}(t)
= \begin{bmatrix}\frac{\mathrm{d}}{\mathrm{d}t} S(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} I(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} R(t)
\\ \frac{\mathrm{d}}{\mathrm{d}t} C(t) \end{bmatrix}
= \begin{bmatrix} \omega R(t) - \big(\frac{a \beta I(t)}{N(t)} + \nu\big) S(t)
\\ \frac{a \beta I(t)}{N(t)} S(t) - \gamma I(t) \\ \gamma
I(t) + \nu S(t)- \omega R(t)
\\ \frac{\beta a I(t)}{N(t)} S(t)
\end{bmatrix}$$

Let's modify the functions `f_model_2` and `model_2` to account for
that:



In [1]:
def f_model_2(x, t, beta, a, omega, gamma, nu):
    S = x[0]
    I = x[1]
    R = x[2]
    # and C = x[3], but we don't need C in the equations below

    N = S + I + R
    dS_dt = omega*R - (a*beta*I/N + nu)*S
    dI_dt = (a*beta*I/N)*S - gamma*I
    dR_dt = gamma*I + nu*S - omega*R
    dC_dt = (a*beta*I/N)*S

    return numpy.array([dS_dt, dI_dt, dR_dt, dC_dt])


def model_2(beta=0.05,
            a=10,
            omega=0.0,
            one_over_gamma=5.0,
            p=0.2,
            VE=0.2,
            initial_S=99.0,
            initial_I=1,
            initial_R=0.0,
            max_time=100):

    nu = -numpy.log(1-p*VE)
    gamma = 1/one_over_gamma

    ts = numpy.linspace(0, max_time, default_resolution)
    x0 = [initial_S, initial_I, initial_R, 0] # notice we start with C(0)=0
    xs = odeint(f_model_2, x0, ts, (beta, a, omega, gamma, nu))

    plot_results(ts, xs, ["S (susceptible)", "I (infectious)", "R (recovered)", "C (cumulative)"])

    R0 = a*beta/gamma
    print("R0 = {:.2f}".format(R0))

    return xs

Everything should be working if we call `model_2` with the default
parameter values:



In [1]:
model_2()

And also if we want to interact with the model, we can just call:



In [1]:
interact(model_2, **default_sliders)


# Model 3: An epidemic model with two age groups



By now you should be familiar with the way to solve ODE models in
python, see if you can understand what each part of the code in the
following code cells does:



In [1]:
def f_model_3(x, t, beta1, beta2, a1, a2, c11, c12, c21, c22, omega, gamma, nu1, nu2):
    S1 = x[0]
    I1 = x[1]
    R1 = x[2]
    S2 = x[3]
    I2 = x[4]
    R2 = x[5]

    N1 = S1 + I1 + R1
    N2 = S2 + I2 + R2

    lambda1 = a1*beta1*(c11*I1/N1 + c12*I2/N2)
    lambda2 = a2*beta2*(c21*I1/N1 + c22*I2/N2)

    dS1_dt = omega*R1 - (lambda1 + nu1)*S1
    dI1_dt = lambda1*S1 - gamma*I1
    dR1_dt = gamma*I1 + nu1*S1 - omega*R1
    dS2_dt = omega*R2 - (lambda2 + nu2)*S2
    dI2_dt = lambda2*S2 - gamma*I2
    dR2_dt = gamma*I2 + nu2*S2 - omega*R2

    return numpy.array([dS1_dt, dI1_dt, dR1_dt, dS2_dt, dI2_dt, dR2_dt])


def model_3(beta1=0.1,
            beta2=0.025,
            a1=15,
            a2=12,
            C11=8.9,
            C12=6.3,
            C21=2.0,
            C22=10.2,
            omega=0.00,
            one_over_gamma=5.0,
            p1=0.0,
            p2=0.0,
            VE1=0.8,
            VE2=0.6,
            initial_S1=2719-1,
            initial_S2=4502-1,
            initial_I1=1,
            initial_I2=1,
            initial_R1=0,
            initial_R2=0,
            max_time=365.0):

    c11 = C11/a1
    c12 = C12/a1
    c21 = C21/a2
    c22 = C22/a2

    nu1 = -numpy.log(1-p1*VE1)
    nu2 = -numpy.log(1-p2*VE2)

    gamma = 1/one_over_gamma

    ts = numpy.linspace(0, max_time, default_resolution)
    x0 = [initial_S1, initial_I1, initial_R1, initial_S2, initial_I2, initial_R2]
    xs = odeint(f_model_3, x0, ts,
                (beta1, beta2, a1, a2, c11, c12, c21, c22, omega, gamma, nu1, nu2))

    plot_results(ts, xs[:, :3], ['S1', 'I1', 'R1'])
    plot_results(ts, xs[:, 3:], ['S2', 'I2', 'R2'])

    R01 = a1*beta1/gamma
    R02 = a2*beta2/gamma

    A = R01*c11
    B = R01*c12
    C = R02*c21
    D = R02*c22

    R0 = (A + D + ((A-D)**2 + 4*B*C)**(1/2))/2

    print("R01: {:.2f}".format(R01))
    print("R02: {:.2f}".format(R02))
    print("R0: {:.2f}".format(R0))

    return xs

In [1]:
model_3()

In [1]:
default_sliders['C11'] = (0, 15, 0.1)
default_sliders['C12'] = default_sliders['C11']
default_sliders['C21'] = default_sliders['C11']
default_sliders['C22'] = default_sliders['C11']
default_sliders['beta1'] = default_sliders['beta']
default_sliders['beta2'] = default_sliders['beta']
default_sliders['a1'] = default_sliders['a']
default_sliders['a2'] = default_sliders['a']
default_sliders['p1'] = default_sliders['p']
default_sliders['p2'] = default_sliders['p']
default_sliders['VE1'] = default_sliders['VE']
default_sliders['VE2'] = default_sliders['VE']
default_sliders['initial_S1'] = (0, 5000)
default_sliders['initial_S2'] = (0, 5000)
default_sliders['initial_I1'] = (0, 5000)
default_sliders['initial_I2'] = (0, 5000)
default_sliders['initial_R1'] = (0, 5000)
default_sliders['initial_R2'] = (0, 5000)

In [1]:
interact(model_3, **default_sliders)


# Optional: Model 4



Model 4 is the same as Model 3, but using Nold's function to generate
the mixing, so we can use the already defined `model_3` function:



In [1]:
def model_4(beta1=0.1,
            beta2=0.025,
            a1=15,
            a2=12,
            epsilon1=0.28,
            epsilon2=0.61,
            omega=0.00,
            one_over_gamma=5.0,
            p1=0.0,
            p2=0.0,
            VE1=0.8,
            VE2=0.6,
            initial_S1=2719-1,
            initial_S2=4502-1,
            initial_I1=1,
            initial_I2=1,
            initial_R1=0,
            initial_R2=0,
            max_time=365.0):

    N1 = initial_S1 + initial_I1 + initial_R1
    N2 = initial_S2 + initial_I2 + initial_R2

    f1 = (1 -epsilon1)*a1*N1/((1-epsilon1)*a1*N1 + (1-epsilon2)*a2*N2)
    f2 = 1 - f1
    c11 = epsilon1 + (1-epsilon1)*f1
    c12 = (1-epsilon1)*f2
    c21 = (1-epsilon2)*f1
    c22 = epsilon2 + (1-epsilon2)*f2

    C11 = c11*a1
    C12 = c12*a1
    C21 = c21*a2
    C22 = c22*a2

    return model_3(beta1=beta1, beta2=beta2, a1=a1, a2=a2, C11=C11,
                   C12=C12, C21=C21, C22=C22, omega=omega,
                   one_over_gamma=one_over_gamma, p1=p1, p2=p2,
                   VE1=VE1, VE2=VE2, initial_S1=initial_S1,
                   initial_S2=initial_S2, initial_I1=initial_I1,
                   initial_I2=initial_I2, initial_R1=initial_R1,
                   initial_R2=initial_R2, max_time=max_time)

In [1]:
model_4()

We need just proper ranges for `epsilon1` and `epsilon2` to



In [1]:
default_sliders['epsilon1'] = (0, 1, 0.01)
default_sliders['epsilon2'] = (0, 1, 0.01)

In [1]:
interact(model_4, **default_sliders)


# Bonus Model 5: Impact of vaccination delay and capacity during an epidemic



With this epidemic model we will study the effect of vaccination
during an epidemic but when we might not have the vaccine available
from the start of the epidemic, or when we might be limited by how
many people can we vaccinate per day. For that we will keep track of
non-vaccinated (superscript ${}^n$) and vaccinated (superscript
${}^v$) individuals in different classes. The system of differential
equations is in this case:

$$\frac{\mathrm{d}}{\mathrm{d}t}\mathbf{x}(t)
= \begin{bmatrix}
\frac{\mathrm{d}}{\mathrm{d}t} S^n(t) \\
\frac{\mathrm{d}}{\mathrm{d}t} I^n(t) \\
\frac{\mathrm{d}}{\mathrm{d}t} R^n(t) \\
\frac{\mathrm{d}}{\mathrm{d}t} S^v(t) \\
\frac{\mathrm{d}}{\mathrm{d}t} I^v(t) \\
\frac{\mathrm{d}}{\mathrm{d}t} R^v(t) \\
\frac{\mathrm{d}}{\mathrm{d}t} C(t)
\end{bmatrix} = \begin{bmatrix} -
a \beta  \frac{I^n(t)+I^v(t)}{N} S^n(t) - V(t) \\
a \beta  \frac{I^n(t)+I^v(t)}{N} S^n(t) - \gamma I^n(t) \\
\gamma I^n(t) \\ -
a \beta \frac{I^n(t)+I^v(t)}{N} S^v(t) + (1-\mathrm{VE}) V(t) \\
a \beta \frac{I^n(t)+I^v(t)}{N} S^v(t) - \gamma I^v(t) \\
\gamma I^v(t) + \mathrm{VE} V(t) \\
a \beta a \frac{I^n(t)+I^v(t)}{N} (S^n(t) + S^v(t))
\end{bmatrix} $$

and the initial condition:

$$\mathbf{x_0}
= \begin{bmatrix} S_0
\\ I_0 \\ R_0 \\ 0 \\ 0 \\ 0 \\ 0\end{bmatrix}$$

where $V(t)$ represents the vaccinations per day at time $t$ and
$\mathrm{VE}$ the vaccine efficacy.

The interesting part in the definition of the model, is how we can
incorporate the time dependent "parameter" vaccinations into the
definition of $f$. For example assuming that you either vaccinate at
the same fixed rate per day or don't vaccinate at all, we could
define:



In [1]:
def f_model_5(x, t, beta, a, VE, gamma, vacc_per_day, start_vacc, duration_vacc):
    Sn = x[0]
    In = x[1]
    Rn = x[2]
    Sv = x[3]
    Iv = x[4]
    Rv = x[5]

    N = Sn + In + Rn + Sv + Iv + Rv

    if t < start_vacc or t > start_vacc + duration_vacc:
        V = 0
    else:
        V = vacc_per_day

    new_infections_n = a*beta*(In+Iv)/N*Sn
    new_infections_v = a*beta*(In+Iv)/N*Sv

    dSn_dt = - new_infections_n - V
    dIn_dt = new_infections_n - gamma*In
    dRn_dt = gamma*In
    dSv_dt = - new_infections_v + (1-VE)*V
    dIv_dt = new_infections_v - gamma*Iv
    dRv_dt = gamma*Iv + VE*V
    dC_dt = new_infections_n + new_infections_v

    return numpy.array([dSn_dt, dIn_dt, dRn_dt, dSv_dt, dIv_dt, dRv_dt, dC_dt])

As you notice with the `if` call above, one of the advantages of the
python programming language is that it was designed to be a highly
readable language. Let's start with a definition of `model_5` that
just computes the results. This should be straight forward by now:



In [1]:
def model_5(beta=0.02,
            a=10,
            VE=0.85,
            one_over_gamma=20.0,
            start_vacc=30,
            vacc_per_day=20,
            duration_vacc=30,
            initial_S=999.0,
            initial_I=1.0,
            initial_R=0.0,
            max_time=365.0):

    ts = numpy.linspace(0, max_time, default_resolution)
    x0 = [initial_S, initial_I, initial_R, 0, 0, 0, 0]
    xs = odeint(f_model_5, x0, ts, (beta, a, VE, 1/one_over_gamma, vacc_per_day, start_vacc, duration_vacc))

    return xs


## Slicing arrays



In the case of models with too many classes, such as Model 5, it is
also important to decide what part to visualize. For example we could
aggregate individuals without paying attention to their vaccination
status. First we need to learn how to select from the results the ones
that we want, this was already hinted at in the definition of
`model_3`.

Remember that the result of `odeint` (and thus of `model_5`) is
matrix:



In [1]:
results_model_5 = model_5()
numpy.shape(results_model_5)

In this case, one with 1,000 rows (the values of $t$) and 7 columns (6
classes and the cumulative infections). To select parts of that matrix
(in NumPy words: to slice the array) we need to indicate which rows
and which columns we want. The basic selection for the case of a
matrix `M` is to use `M[n, m]` where you are indicating to select the
n-th row and m-th column. By the way, it is very important to remember
that python starts numbering with 0. So in order to select from our
results the number of susceptible vaccinated individuals for the last
time available (after one year), we would use:



In [1]:
results_model_5[999, 3]

Another way to slice an array is to select ranges. You can do it using
`[from_row:to_row, from_column:to_column]`, where you indicate with
`from_row` the first row that you want to include (remember the first
row is 0) and with `to_row` the first row that you do not want to
include. In other words you go `from_row` `to_row` not including the
last one. Similarly for columns.

Perhaps it is much clearer with an example:



In [1]:
results_model_5[0:500, 1:3]

This would select the first 500 values of $t$ (half a year because
`max_time` is 365.0 and `default_resolution` is 1,000) for the
non-vaccinated infectious individuals and the non-vaccinated recovered
individuals.

If you omit the first part of the selection in `n:m`, that is you
write instead `:m`, 0 would be assumed as the value for `n`. If you
omit the last one, that is if you write `n:` then the size of the
array will be assumed as `m`. So the following code cell will give
exactly the same as the previous code cell:



In [1]:
results_model_5[:500, 1:3]

You can also omit both and you can also combine both ways to slice the
array. For example to obtain the results for all times for the
non-vaccinated susceptible individuals you would use:



In [1]:
results_model_5[:, 0]

To obtain the results for half a year after the epidemic started for
all classes you would use:



In [1]:
results_model_5[499, :]

Using `results_model_5[499]` would also work. Notice that if we do
that then we obtain vectors:



In [1]:
numpy.shape(results_model_5[:, 0])

In [1]:
numpy.shape(results_model_5[499])


## Visualizing Model 5



So now that we know how to select parts of the results, we can
visualize the results that we want. As stated before, one possibility
is to aggregate non-vaccinated and vaccinated individuals:



In [1]:
def model_5(beta=0.02,
            a=10,
            VE=0.85,
            one_over_gamma=20.0,
            start_vacc=30,
            vacc_per_day=20,
            duration_vacc=30,
            initial_S=999.0,
            initial_I=1.0,
            initial_R=1.0,
            max_time=365.0):

    ts = numpy.linspace(0, max_time, default_resolution)
    x0 = [initial_S, initial_I, initial_R, 0, 0, 0, 0]
    xs = odeint(f_model_5, x0, ts, (beta, a, VE, 1/one_over_gamma, vacc_per_day, start_vacc, duration_vacc))

    total_S = xs[:, 0] + xs[:, 3]
    total_I = xs[:, 1] + xs[:, 4]
    cum_infected = xs[:, 6]
    vaccinated = xs[:, 3] + xs[:, 4] + xs[:, 5]

    ys = numpy.vstack([total_S, total_I, cum_infected, vaccinated]).T

    plot_results(ts, ys, ['Total susceptible', 'Total infectious', 'Cumulative infected', 'Vaccinated'])

    return xs

The only new method used here is `numpy.vstack`, you should realize
how it works just by knowing that it stands for vertically stacking,
so it puts stacks vectors one on top of each other and we obtain a
matrix, whose rows are the vectors that we give to the function. The
only problem is that in this case it is a matrix with 4 rows and 1,000
columns, to use it for `plot_results` (or for `pyplot.plot`) we want
the results the other way around, with 1,000 rows and 4 columns. We
just need to transpose the matrix, and that is exactly what `.T` does.

If you want to visualize other aggregations, try to modify the code
above and run it again.

As usual to use `interact`, we just need to add the new parameters:



In [1]:
default_sliders['start_vacc'] = (0, 30*11, 5)
default_sliders['vacc_per_day'] = (0, 800, 10)
default_sliders['duration_vacc'] = (1, 30*6, 5)

In [1]:
interact(model_5, **default_sliders)

Should there be a limit to the variables of vaccinations per day or
the variable of duration of vaccination? What is happening when you
set them to high? If you cannot identify what is happening substitute
the line `plot_results(...` with `pyplot.plot(ts, total_S)` in the
`model_5` definition above and rerun that code cell and the last code
cell.

